In [ ]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
os.environ["OPENAI_API_KEY"] = "sk-proj-FZ2v2sIScsxguqHnuT4VT3BlbkFJEgAuDrqMx91v7PG2fhDu"
django.setup()
from apps.email_manager.service_layer.email_linker import ConversationReader
import json
from datetime import datetime
from apps.email_manager.service_layer.insurance_erp_llm import InsuranceERPLLM
from apps.email_manager.models.email import EmailAttachment



In [ ]:
convo_reader = ConversationReader(
    start_date=datetime(year=2024, month=2, day=10),
    end_date=datetime(year=2024, month=2, day=20),
)
convo_reader.extract_conversation()


In [ ]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'ISSUE POLICY FOR EEI | M/S SIKA PAKISTAN ' in a_convo_str:
        (full_convo, attachments, all_files) = a_convo_data.get_full_convo()
        attachments: list[EmailAttachment] = attachments
        insLLM = InsuranceERPLLM()
        insLLM.store_convo_in_vector_store(f"{full_convo}")
        insLLM.search_convo_vector_store()
        insLLM.query_from_final_results(insLLM.convo_query_results)
        print("------------------------------------")
        insLLM.store_documents_in_vector_store(attachments)
        insLLM.search_documents_vector_store()
        insLLM.query_from_final_results(insLLM.document_query_results)
        insLLM.clean_query_results()
        for attchmnt in attachments:
            print(attchmnt)
        # for a_msg in full_convo:
        #     print(json.dumps(a_msg, indent=4))
        insLLM.record_data(all_files)
        print("")
        print("------------------------------------")
        print("")
        break


In [ ]:
from datetime import datetime
from os import environ
import re
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
    SystemMessage,
    AIMessage
)
import json
import torch
from transformers import pipeline
from statistics import mode

from apps.risk_manager.models.policy import Policy, PolicyFile
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

from db_tables import Base, SQLPolicyFile

In [ ]:
print(environ.get("DB_HOST"))
DATABASE_URL = f'postgresql://{environ.get("POSTGRES_USER")}:{environ.get("POSTGRES_PASSWORD")}@{environ.get("DB_HOST")}:{environ.get("DB_PORT")}/{environ.get("POSTGRES_DB")}'

engine = create_engine(DATABASE_URL)

Session = sessionmaker(bind=engine)
session = Session()
if insLLM.event_type == 'policy_issued':
    new_policy = Policy.objects.create(
        issue_date=insLLM.queries_results["issue_date"]["value"],
        number=insLLM.queries_results["policy_number"]["value"],
        net_premium=insLLM.queries_results["net_premium"]["value"],
    )
    # /Users/mirbilal/Desktop/minsir/media/email_attachments/ADAMJEE_INSURANCE_PAYMENT_SWIFT_16.02.2024-2_ZrruCMA.pdf
    for file_path in attachments:
        final_path = file_path.replace("/Users/mirbilal/Desktop/minsir/media/", "")
        name = final_path.replace("email_attachments/", "")
        file = SQLPolicyFile(
            policy_id=new_policy.id,
            name = name,
            file = final_path
        )
        session.add(file)
    session.commit()

In [ ]:
# print(json.dumps(insLLM.convo_query_results, indent=4))
print("--------")
# print(json.dumps(insLLM.document_query_results, indent=4))
print("--------")
# print(json.dumps(insLLM.raw_results, indent=4))
# print("--------")
print(json.dumps(insLLM.queries_results, indent=4))

In [ ]:
print(insLLM.queries_results)

In [ ]:
from datetime import datetime
import re


for key, value_dict in insLLM.queries_results.items():
    data_type = value_dict["type"]
    data_value = value_dict["value"]
    # value_dict["value"] = insLLM.raw_results[key]
    # continue
    if data_value is not None and type(data_value) is str and "404 Not Found" not in data_value:
        # Clean the data value
        data_values = data_value.split("=")
        cleaned_value = data_values[-1].strip().replace("'", "")

        # Convert the cleaned data value to the specified type
        if data_type == "double":
            try:
                # Extract numbers from the string using regex
                cleaned_value = re.sub(r'[^\d.]', '', cleaned_value)
                cleaned_value = float(cleaned_value)
            except ValueError:
                cleaned_value = None
                raise Exception
        elif data_type == "datetime":
            try:
                # Extract date in the format YYYY-MM-DD using regex
                match = re.search(r'\d{4}-\d{2}-\d{2}', cleaned_value)
                if match:
                    cleaned_value = datetime.strptime(match.group(), "%Y-%m-%d")
                else:
                    cleaned_value = None
            except ValueError:
                cleaned_value = None
                raise Exception
        elif data_type == "string":
            cleaned_value = str(cleaned_value)
        
        # Update the value in the dictionary
        value_dict["value"] = cleaned_value

print(insLLM.queries_results)

In [ ]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'ISSUE POLICY FOR EEI | M/S SIKA PAKISTAN ' in a_convo_str:
        (full_convo, attachments) = a_convo_data.get_full_convo()
        attachments: list[EmailAttachment] = attachments
        # insLLM = InsuranceERPLLM()
        # insLLM.store_documents_in_vector_store(attachments)
        # insLLM.search_documents_vector_store()
        # insLLM.query_from_final_results(insLLM.document_query_results)
        for attchmnt in attachments:
            print(attchmnt)
        for a_msg in full_convo:
            print(json.dumps(a_msg, indent=4))
        print("")
        print("------------------------------------")
        print("")
        break

In [ ]:
insLLM = InsuranceERPLLM()
insLLM.store_convo_in_vector_store(f"{dummy_convo}")
insLLM.search_convo_vector_store()
insLLM.query_from_final_results(insLLM.convo_query_results)